In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from typing import List

from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_text_splitters import TokenTextSplitter
from neo4j.exceptions import ClientError

import openai
from openai import OpenAI
from openai import AzureOpenAI

from time import sleep



In [ ]:
# Load environment variables
load_dotenv(r"C:\Users\lriospie\OneDrive - azureford\Documents\Maestria\Integrador\Neo4j_RAG\.env",verbose=True)


In [ ]:
# client = AzureOpenAI(
#     api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
#     api_version="2023-05-15",
#     azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# )


AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY"), 

#deployment_name='text-embedding-3' #This will correspond to the custom name you chose for your deployment when you deployed a model.

# Embeddings & LLM models
embedding_dimension = 1536
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3",api_version="2024-02-01",dimensions=embedding_dimension)

llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0)

# Get Neo4j credentials from environment variables
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USER")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph()
sleep(5)
graph.query("MATCH (n) DETACH DELETE n")

In [ ]:
# Load the text file
txt_path ="C:/Users/lriospie/OneDrive - azureford/Documents/Maestria/Integrador/Neo4j_RAG/Entrega Modelo Final/sec_10K_data/10_K_MSFT_0000950170-23-035122.txt"
#loader = TextLoader(str(txt_path))
loader = TextLoader(str(txt_path), encoding='utf-8')
documents = loader.load()

# Ingest Parent-Child node pairs
parent_splitter = TokenTextSplitter(chunk_size=512*3, chunk_overlap=24)
child_splitter = TokenTextSplitter(chunk_size=100*3, chunk_overlap=24)
parent_documents = parent_splitter.split_documents(documents)

In [ ]:
graph.query("MATCH (n) DETACH DELETE n")

In [ ]:
for i, parent in enumerate(parent_documents):
    child_documents = child_splitter.split_documents([parent])
    params = {
        "parent_text": parent.page_content,
        "parent_id": i,
        "parent_embedding": embeddings.embed_query(parent.page_content),
        "children": [
            {
                "text": c.page_content,
                "id": f"{i}-{ic}",
                "embedding": embeddings.embed_query(c.page_content),
            }
            for ic, c in enumerate(child_documents)
        ],
    }
    # Ingest data
    sleep(5)
    graph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    SET p.text = $parent_text
    WITH p
    CALL db.create.setVectorProperty(p, 'embedding', $parent_embedding)
    YIELD node
    WITH p 
    UNWIND $children AS child
    MERGE (c:Child {id: child.id})
    SET c.text = child.text
    MERGE (c)<-[:HAS_CHILD]-(p)
    WITH c, child
    CALL db.create.setVectorProperty(c, 'embedding', child.embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index for child
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('parent_document', "
            "'Child', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  # already exists
        pass
    # Create vector index for parents
    try:
        graph.refresh_schema()
        graph.query(
            "CALL db.index.vector.createNodeIndex('typical_rag', "
            "'Parent', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  # already exists
        pass


In [ ]:
# Ingest hypothethical questions
class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on " "the information from the text"
        ),
    )


questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating hypothetical questions based on the information "
                "found in the text and must reply in JSON format. Make sure to provide full context in the generated "
                "questions."
            ),
        ),
        (
            "human",
            (
                "Use the given format to generate hypothetical questions from the"
                "following input: {input}"
            ),
        ),
    ]
)

question_chain = questions_prompt | llm.with_structured_output(Questions.json)

In [ ]:
def clean_questions_data(questions_data):
    if 'questions' not in questions_data:
        return []

    cleaned_questions = []
    for entry in questions_data['questions']:
        if isinstance(entry, dict) and 'question' in entry:
            cleaned_questions.append(entry)
        elif isinstance(entry, str):
            cleaned_questions.append({'question': entry})
        else:
            # Handle cases where the entry is not a string or doesn't contain the 'question' key
            print(f"Invalid entry found and skipped: {entry}")
    # Convert the cleaned_questions list into a pandas DataFrame
    df = pd.DataFrame(cleaned_questions)

    # Generate a unique file ID using the current timestamp
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    file_id = f"questions_{timestamp}"

    # Save the DataFrame into a CSV file with the unique file ID
    df.to_csv(f"{file_id}.csv", index=False)

    return cleaned_questions

for i, parent in enumerate(parent_documents):
    questions_data = question_chain.invoke(parent.page_content)
    cleaned_questions_list = clean_questions_data(questions_data)
    print(cleaned_questions_list)
    if 'questions' in questions_data:
        questions_list = questions_data['questions']  # Access the 'questions' list
        params = {
            "parent_id": i,
            "questions": []
        }
        
        for iq, q in enumerate(cleaned_questions_list):  # Use the cleaned questions list
            if isinstance(q, dict) and 'question' in q:
                try:
                    question_text = q['question']
                    embedding = embeddings.embed_query(question_text)
                    params["questions"].append({"text": question_text, "id": f"{i}-{iq}", "embedding": embedding})
                except Exception as e:
                    print(f"Error embedding question {q}: {e}")
            else:
                print(f"Skipping invalid question entry: {q}")
        
        sleep(5)
        graph.query(
            """
        MERGE (p:Parent {id: $parent_id})
        WITH p
        UNWIND $questions AS question
        CREATE (q:Question {id: question.id})
        SET q.text = question.text
        MERGE (q)<-[:HAS_QUESTION]-(p)
        WITH q, question
        CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
        YIELD node
        RETURN count(*)
        """,
            params,
        )
        # Create vector index
        try:
            graph.refresh_schema()
            graph.query(
                "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
                "'Question', 'embedding', $dimension, 'cosine')",
                {"dimension": embedding_dimension},
            )
        except ClientError:  # already exists
            pass



In [ ]:
# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating concise and accurate summaries based on the "
                "information found in the text. Do not miss in the summaries any information" 
                "related with numbers such as dates, costs, rates, porcentages or any sort of numerical values "
            ),
        ),
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm

In [ ]:
for i, parent in enumerate(parent_documents):
    summary = summary_chain.invoke({"question": parent.page_content}).content
    params = {
        "parent_id": i,
        "summary": summary,
        "embedding": embeddings.embed_query(summary),
    }
    
    sleep(5)
    graph.query(
        """
    MERGE (p:Parent {id: $parent_id})
    MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
    SET s.text = $summary
    WITH s
    CALL db.create.setVectorProperty(s, 'embedding', $embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('summary', "
            "'Summary', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  # already exists
        pass


In [ ]:
from langchain_community.vectorstores import Neo4jVector



# Typical RAG retriever

typical_rag = Neo4jVector.from_existing_index(
    embeddings, index_name="typical_rag"
)

# Parent retriever

parent_query = """
MATCH (node)<-[:HAS_CHILD]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata LIMIT 1
"""

parent_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="parent_document",
    retrieval_query=parent_query,
)

# Hypothetic questions retriever

hypothetic_question_query = """
MATCH (node)<-[:HAS_QUESTION]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

hypothetic_question_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="hypothetical_questions",
    retrieval_query=hypothetic_question_query,
)
# Summary retriever

summary_query = """
MATCH (node)<-[:HAS_SUMMARY]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

summary_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="summary",
    retrieval_query=summary_query,
)

In [ ]:
response = hypothetic_question_vectorstore.similarity_search(
    "What is The Department of Justice recommendation on marijuana"
)
print(response[0].page_content)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

graph.refresh_schema()

vector_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=typical_rag.as_retriever()
)

vector_qa.invoke(
     "Which other drugs appear along with marijuana in the Schedule I"
)

In [ ]:
graph.refresh_schema()

vector_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=typical_rag.as_retriever()
)

vector_qa.invoke(
    #"What Wexler previously told to USA TODAY"
    "What experts  exposed to USA TODAY on marijuana"
)

In [ ]:
from langchain.chains import GraphCypherQAChain

# graph = Neo4jGraph()
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = AzureChatOpenAI(azure_deployment='Chat_gpt_4',api_version="2023-05-15", temperature=0),
    qa_llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0), graph=graph, verbose=True,
)

In [ ]:
graph.refresh_schema()
cypher_chain.invoke(
     "How many times Schedule III is mentioned within the text"
)

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

graph = Neo4jGraph()
graph.refresh_schema()

In [ ]:
tools = [
    Tool(
        name="Tasks",
        func=vector_qa.run,
        description="""Useful to reply to most of the queries as long as any sort of aggregation is not requested.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.run,
        description="""Usfeful when any sort of aggregation is requested in the query.
        Use full question as input.
        """,
    ),
]

mrkl = initialize_agent(
    tools, 
    AzureChatOpenAI(azure_deployment='Chat_gpt_4',api_version="2023-05-15", temperature=0),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)


In [ ]:
response = mrkl.invoke("How many times Schedule III is mentioned within the text")
print(response)

In [ ]:
response = mrkl.invoke("Who might be affected with IQ loss due to marijuana consumption")
print(response)